In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
# from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-annotated/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'labels','commit_message':'message','commit_diff':'diffs'})
df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

,labels,diffs,message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_DEL_LINE,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE
commit_sha,,,,,,,,,,,,,,,,,,,,,
be937a3290223f926fe50684f1344569a573ed4b,C,diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
c800d2e36954edddcb83aa1df7f623f2780c7780,A,diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
469e56eeff17eb857c95e935ca7b49723c43470e,P,diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...,0,0,0,0,0,0,0,...,0,0,8,0,0,15,0,0,0,0
24f5d302c0906f3377f4d6bcc2735d943ce13e3d,A,diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,...",3,37,0,0,9,0,0,...,0,0,0,0,0,0,1,2,0,0
1effdb7b959503596b07f7f3e74618ab63f9e5e5,P,diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...,1,0,12,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,C,diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,C,diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...,2,6,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,C,diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...,1,1,5,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [5]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/eval.csv',index=False)

In [6]:
len(train)

1106

In [7]:
train['labels'].value_counts()

labels
P    691
C    242
A    173
Name: count, dtype: int64

In [8]:
test['labels'].value_counts()

labels
P    155
C     51
A     32
Name: count, dtype: int64

In [9]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [10]:
train['embeddings'] = train['diffs'].apply(model.encode)
test['embeddings'] = test['diffs'].apply(model.encode)

In [11]:
test

,labels,diffs,message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE,embeddings
commit_sha,,,,,,,,,,,,,,,,,,,,,
439b7750d4750722a03d60465da2e4b62bbcf626,C,diff --git a/org.springframework.orm/src/main/...,"fixed ""hibernateManagedSession"" mode to actual...",1,5,0,1,5,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.08523224, 0.47531417, 0.008388178, 0.03423..."
67817d4e45fe5640c971403dbde8be347b71f12a,P,diff --git a/src/cascadia/TerminalSettingsMode...,Consolidate IInheritable macros (#11924)\n\nCo...,1,14,69,8,13,21,3,...,0,0,0,0,0,0,0,0,0,"[-0.12605347, 0.07784113, -0.028077677, 0.0561..."
f196dd5ff7871118eb42738f1ef5f771b0b9529c,A,diff --git a/modules/imgproc/include/opencv2/i...,preliminary version of a multi-level findconto...,3,231,16,28,16,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.08439066, -0.06631157, 0.005969448, 0.0884..."
75c8e09d6d412e1012a921d9f3932d6f253781e6,A,diff --git a/crates/swc_css_codegen/src/writer...,feat(api/rust): Implement more traits for publ...,4,6,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.0833264, 0.32967168, -0.05786188, 0.002498..."
707ee709663ce7e8faac5345d4007f205379187a,P,diff --git a/src/training/pango_font_info.cpp ...,Use deprecated pango_fc_font_get_glyph for old...,1,4,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.04197974, 0.18034181, 0.044548623, 0.05202..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8b25f44f0570c7b430d632108126a91f1939de65,P,diff --git a/Cargo.lock b/Cargo.lock\nindex f7...,build(deps): bump toml from 0.6.0 to 0.7.0 (#5...,0,0,0,0,0,0,0,...,0,0,0,0,0,6,0,0,11,"[-0.10873181, 0.29730272, -0.0186197, 0.052268..."
850555d1faa9be7d8306adffd95c7dee5e58717f,C,diff --git a/src/core/util/next-tick.js b/src/...,fix: always use microtasks for nextTick (#8450...,3,17,29,17,19,9,15,...,0,0,0,0,0,0,0,0,0,"[-0.06421692, -0.16394633, -0.057304293, 0.087..."
7dfdb815e40d79e857f5d2e6a7c1519428bbf058,P,diff --git a/.git-blame-ignore-revs b/.git-bla...,[docs] Add warning about git-blame-ignore-revs...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,3,0,0,"[-0.18914664, -0.17303085, 0.07548259, 0.05755..."


In [12]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['labels'].to_list()
y_test = test['labels'].to_list()

In [13]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [14]:
from sklearn import metrics
predicted = LR.predict(X_test)

print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.6322792274804658
Logistic Regression Recall: 0.6638655462184874
Logistic Regression Recall: 0.6346237744553788
Logistic Regression Accuracy: 0.6638655462184874
